In [16]:
# Imports necesarios
import os
import platform
import shutil
import zipfile

import pandas as pd
import requests
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

EXCEL_FILE = 'Tabla.xlsx'
COLUMN = 'GenBank assembly accession'
ACCESSIONS_FILE = 'salida.fasta'
GENOMES_FOLDER = 'secuencias'


La primera parte consiste en leer el archivo Tabla.xlsx para poder leer las secuencias que se utilizaron de NBCI

In [7]:
# Archivo excel del que vamos a extraer los identificadores de
# las secuencias de ADN que se van a descargar

archivo_excel = 'Tabla.xlsx'
df = pd.read_excel(archivo_excel)

# Nombre de la columna que contiene las secuencias de ADN
columna = 'GenBank assembly accession'

# Se crean objetos SeqRecord para cada secuencia y se guardan en una lista
obj = []
for index, row in df.iterrows():
    sequence = str(row[columna])
    if pd.notna(sequence):
        record = SeqRecord(Seq(sequence), id=str(index), description="")
        obj.append(record)

# Archivo .fasta resultante
archivo_fasta = 'salida.fasta'

# Se guardan los registros en un archivo .fasta
with open(archivo_fasta, "w") as fasta_file:
    for secuencia in obj:
        if pd.notna(secuencia.seq):
            fasta_file.write(str(secuencia.seq) + "\n")


Esto lo podemos conseguir de la siguiente manera, procederemos a descargar los archivos en un directorio llamado
"secuencias", por medio de una petición GET a la API de NCBI

In [9]:
if not os.path.exists(GENOMES_FOLDER):
    os.makedirs(GENOMES_FOLDER)

with open('salida.fasta') as f:
    accessions = f.readlines()
    accessions = [x.strip() for x in accessions]
    accessions = filter(lambda x: x != '' and x != 'nan', accessions)

    for accession in accessions:
        url = f'https://api.ncbi.nlm.nih.gov/datasets/v2alpha/genome/accession/{accession}/download?include_annotation_type=GENOME_FASTA'
        r = requests.get(url)

        with open(f'{GENOMES_FOLDER}/{accession}.zip', 'wb') as fasta:
            fasta.write(r.content)
        print(f'Secuencia {accession} descargada con éxito.')
        print(f'Extrayendo el zip')

        with zipfile.ZipFile(f'{GENOMES_FOLDER}/{accession}.zip', 'r') as zip_ref:
            zip_ref.extractall(GENOMES_FOLDER)

        print(f'Secuencia {accession} extraída con éxito.')
        print(f'Eliminando el zip')
        os.remove(f'{GENOMES_FOLDER}/{accession}.zip')



Secuencia GCA_000681515.2 descargada con éxito.
Extrayendo el zip
Secuencia GCA_000681515.2 extraída con éxito.
Eliminando el zip
Secuencia GCA_000987705.1 descargada con éxito.
Extrayendo el zip
Secuencia GCA_000987705.1 extraída con éxito.
Eliminando el zip
Secuencia GCA_001317555.2 descargada con éxito.
Extrayendo el zip
Secuencia GCA_001317555.2 extraída con éxito.
Eliminando el zip
Secuencia GCA_001342195.2 descargada con éxito.
Extrayendo el zip
Secuencia GCA_001342195.2 extraída con éxito.
Eliminando el zip
Secuencia GCA_001463975.1 descargada con éxito.
Extrayendo el zip
Secuencia GCA_001463975.1 extraída con éxito.
Eliminando el zip
Secuencia GCA_001465135.2 descargada con éxito.
Extrayendo el zip
Secuencia GCA_001465135.2 extraída con éxito.
Eliminando el zip
Secuencia GCA_001466165.1 descargada con éxito.
Extrayendo el zip
Secuencia GCA_001466165.1 extraída con éxito.
Eliminando el zip
Secuencia GCA_001466195.1 descargada con éxito.
Extrayendo el zip
Secuencia GCA_001466195.

In [18]:
# Creamos un nuevo directorio llamado data
if not os.path.exists('data'):
    os.makedirs('data')

# We get the absolute path of the current directory
data_dir = os.path.join(os.getcwd(), 'data')

# Copiamos todos los archivos dentro de la carpeta secuencia que están 
# en formato .fna a la carpeta data de manera recursiva, esto es, 
# incluyendo los archivos en los subdirectorios de secuencias.

for root, dirs, files in os.walk(GENOMES_FOLDER):
    for file in files:
        if file.endswith('.fna'):
            print(f'Copiando {file} a la carpeta {data_dir}')
            # Lo copiamos a la carpeta data
            shutil.copyfile(os.path.join(root, file), os.path.join(data_dir, file))


Copiando GCA_000681515.2_ASM68151v2_genomic.fna a la carpeta C:\Users\Misael\Documents\github\materias\GC-20241\exposicion\resources\code\data
Copiando GCA_000987705.1_ASM98770v1_genomic.fna a la carpeta C:\Users\Misael\Documents\github\materias\GC-20241\exposicion\resources\code\data
Copiando GCA_001317555.2_ASM131755v2_genomic.fna a la carpeta C:\Users\Misael\Documents\github\materias\GC-20241\exposicion\resources\code\data
Copiando GCA_001342195.2_ASM134219v2_genomic.fna a la carpeta C:\Users\Misael\Documents\github\materias\GC-20241\exposicion\resources\code\data
Copiando GCA_001463975.1_ASM146397v1_genomic.fna a la carpeta C:\Users\Misael\Documents\github\materias\GC-20241\exposicion\resources\code\data
Copiando GCA_001465135.2_ASM146513v2_genomic.fna a la carpeta C:\Users\Misael\Documents\github\materias\GC-20241\exposicion\resources\code\data
Copiando GCA_001466165.1_ASM146616v1_genomic.fna a la carpeta C:\Users\Misael\Documents\github\materias\GC-20241\exposicion\resources\code

## Alineamiento de secuencias

Para el alineamiento de secuencias usaremos el programa MAFFT, en particular la versión 7.520, que se puede descargar de la siguiente
página: https://mafft.cbrc.jp/alignment/software/windows.html

Una vez instalado el programa, nos movemos a la carpeta `data` que es donde pusimos todos los archivos fasta, hecho esto podemos alinear 
las secuencias de la siguiente manera:

Primero alineamos una sola secuencia, por poner un ejemplo:

`mafft --auto GCA_000681515.2_ASM68151v2_genomic.fna > first_align.fasta`

Después alineamos todas las secuencias con respecto a dicha secuencia, es decir:

`mafft --6merpair --thread -1 --keeplength --addfragments *.fna first_align.fasta > alineamiento.fasta`



In [ ]:
# Mandamos a llamar al comando wsl desde python
# para poder ejecutar el comando de alineamiento
other_sequences = os.listdir('data')
other_sequences = list(filter(lambda x: x.endswith('.fna'), other_sequences))
others = ','.join(other_sequences)

PRIMER_COMANDO = 'mafft --auto GCA_000681515.2_ASM68151v2_genomic.fna > first_align.fasta'
SEGUNDO_COMANDO = f'mafft --6merpair --thread -1 --keeplength --addfragments {others} first_align.fasta > alineamiento.fasta'

# Si se está ejecutando desde Windows concatenamos al principio el comando wsl
if platform.system() == 'Windows':
    PRIMER_COMANDO = 'wsl.exe ' + PRIMER_COMANDO
    SEGUNDO_COMANDO = 'wsl.exe ' + SEGUNDO_COMANDO

os.system(PRIMER_COMANDO)
os.system(SEGUNDO_COMANDO)

## PCA (Principal Component Analysis)

Usaremos TASSEL para hacer el PCA, se trata de una herramienta hecha en Java de análisis estadístico de datos genéticos que
permite evaluar asociaciones de rasgos, patrones evolutivos y desequilibrio de ligamiento, entre otras cosas...

Empezamos por abrir el programa, después de hacerlo nos encontraremos con la siguiente pantalla:

![TASSEL](img/tassel_1.png)

Procedemos a cargar el archivo ``alineamiento.fasta`` que generamos en el paso anterior.

Para ello hacemos click en el menú File -> Open y seleccionamos el archivo ``alineamiento.fasta`` que se encuentra en la carpeta data

Una vez abierto el archivo, verémos algo como esto:

![TASSEL](img/tassel_2.png)


Siguiendo las instrucciones de este vídeo [PCA Analysis with TASSEL](https://www.youtube.com/watch?v=PHmaszKtxWM) podemos hacer el PCA.

Primero tenemos que efectuar un control de calidad sobre los genotipos, para ello hacemos click en el menú Filter -> Filter Genotype Table.

Abierta esa ventana establecemos el valor de Site Min Count al número de secuencias que tenemos, en nuestro caso son 504, así que ponemos ese valor.

Además establecemos el valor de Site Min Frequency a 0.05, esto es, que el alelo debe estar presente en al menos el 5% de las secuencias.





## Identificación de genes específicos de las cepas

## Anotaciones en los genomas

Para las anotaciones de los genomas, usaremos el programa Prokka, el cual es un programa de anotación de genomas de procariotas
que produce archivos de anotación en GenBank, EMBL o formato GFF3, y archivos de secuencia de proteínas en formato FASTA.
En nuestro caso queremos generar los archivos en formato GFF3, para después generar un pangenoma con Roary.

## Pangenoma

## Resultados


<img src="img/clusters_and_tree.jpg" alt="Clusters y árbol filogenético" width="1080" height="auto">

<img src="img/relative_abundance.jpg" alt="Abundancia relativa" width="1080" height="auto">